# Swift DeepSky over Stripe82 cleaning

We have two tables -- count rates and nufnu fluxes -- containing the combined results of the SDS pipeline over all Swift observations within the Stripe82 region.

Most probably these tables contain duplicated entries, which we want to remove.
The catalog we want is to be made by unique sources, usually called *primary* sources, that score better is a given property.

The criterion we will use to select filter the sources is their signal-to-noise ratio in the `full` band: $0.3-10 keV$; *i.e*, all detected events.
The object that scores better among its duplicates guarantees its place in the final catalog.

The cross-matching and filtering process is done by `xmatch`, we give it the coordinates and selection criterion columns and it outputs the primary catalog.

## Filter catalogs

Filenames:
* `table_countrates_detections_stripe82.csv`
* `table_flux_detections_stripe82.csv`

In [4]:
import warnings
warnings.simplefilter("ignore")

data_dir = 'swift_deepsky/'
def fname(name):
    from os import path
    return path.join(data_dir, name)

### Energy flux table

In [2]:
!head -n2 $data_dir/table_flux_detections_stripe82.csv

RA;DEC;NH;ENERGY_SLOPE;ENERGY_SLOPE_ERROR;EXPOSURE_TIME;nufnu_3keV(erg.s-1.cm-2);nufnu_error_3keV(erg.s-1.cm-2);nufnu_0.5keV(erg.s-1.cm-2);nufnu_error_0.5keV(erg.s-1.cm-2);upper_limit_0.5keV(erg.s-1.cm-2);nufnu_1.5keV(erg.s-1.cm-2);nufnu_error_1.5keV(erg.s-1.cm-2);upper_limit_1.5keV(erg.s-1.cm-2);nufnu_4.5keV(erg.s-1.cm-2);nufnu_error_4.5keV(erg.s-1.cm-2);upper_limit_4.5keV(erg.s-1.cm-2)
00:56:24.480;-01:16:38.317;3.54E+20;0.8;-999/-999;4572.8;7.96961e-14;2.1177e-14;2.24733e-14;1.55274e-14;-9.990E+02;1.08826e-13;4.02727e-14;-9.990E+02;9.37007e-14;4.60196e-14;-9.990E+02


In [3]:
def run_xmatch(df, flux_column, flux_error_column):

    from astropy.coordinates import SkyCoord
    from astropy import units
    coords = SkyCoord(df['RA'], df['DEC'], unit=(units.hourangle,units.degree))
    ra = coords.ra
    ra.wrap_angle = 180 * units.deg
    dec = coords.dec
    df['RA'] = ra.deg
    df['DEC'] = dec.deg

    df['snr'] = df[flux_column] / df[flux_error_column]
    
    from xmatch import xmatch
    cols = dict(ra='RA', dec='DEC', id='OBJID')

    from astropy.coordinates import Angle
    rad = Angle(5,'arcsec')

    xcat = xmatch(df, df, cols, cols, radius=rad, snr_column='snr')
    
    return xcat

In [6]:
detections_filename = fname('table_flux_detections_stripe82.csv')
flux_column = 'nufnu_3keV(erg.s-1.cm-2)'
flux_error_column = 'nufnu_error_3keV(erg.s-1.cm-2)'

import pandas
df = pandas.read_csv(detections_filename, sep=';')
df.index.name = 'OBJID'
df = df.reset_index()
df.head()

,OBJID,RA,DEC,NH,ENERGY_SLOPE,ENERGY_SLOPE_ERROR,EXPOSURE_TIME,nufnu_3keV(erg.s-1.cm-2),nufnu_error_3keV(erg.s-1.cm-2),nufnu_0.5keV(erg.s-1.cm-2),nufnu_error_0.5keV(erg.s-1.cm-2),upper_limit_0.5keV(erg.s-1.cm-2),nufnu_1.5keV(erg.s-1.cm-2),nufnu_error_1.5keV(erg.s-1.cm-2),upper_limit_1.5keV(erg.s-1.cm-2),nufnu_4.5keV(erg.s-1.cm-2),nufnu_error_4.5keV(erg.s-1.cm-2),upper_limit_4.5keV(erg.s-1.cm-2)
0,0,00:56:24.480,-01:16:38.317,3.540000e+20,0.8,-999/-999,4572.8,7.969610e-14,2.117700e-14,2.247330e-14,1.552740e-14,-999.0,1.088260e-13,4.027270e-14,-999.0,9.370070e-14,4.601960e-14,-999.0
1,1,00:56:19.136,-01:14:58.198,3.520000e+20,0.8,-999/-999,4684.2,6.277090e-14,1.975260e-14,1.277270e-14,1.252210e-14,-999.0,7.075490e-14,3.430380e-14,-999.0,1.066430e-13,5.170340e-14,-999.0
2,2,00:56:23.004,-01:13:39.516,3.510000e+20,0.8,-999/-999,4649.6,5.501970e-14,1.974560e-14,2.238590e-14,1.751890e-14,-999.0,4.655630e-14,3.005940e-14,-999.0,9.353370e-14,5.227960e-14,-999.0
3,3,00:56:16.418,-01:14:05.418,3.510000e+20,0.8,-999/-999,4642.0,8.173270e-14,2.256640e-14,1.151060e-14,1.187720e-14,-999.0,9.568910e-14,3.936350e-14,-999.0,1.442380e-13,5.933510e-14,-999.0
4,4,00:56:16.922,-01:13:16.401,3.500000e+20,0.8,-999/-999,4587.3,7.365840e-14,2.115000e-14,5.616880e-14,2.418840e-14,-999.0,4.673000e-14,2.649940e-14,-999.0,9.390950e-14,4.601620e-14,-999.0


In [5]:
xcat = run_xmatch(df, flux_column, flux_error_column)
xcat.head()

A                          B                        AB             \
          RA       DEC OBJID         RA       DEC OBJID       snr duplicates   
0  14.102000 -1.277310     0  14.102000 -1.277310   0.0  3.763333       3309   
1  14.079733 -1.249499     1  14.079733 -1.249499   1.0  3.177855       3310   
2  14.095850 -1.227643     2  14.095850 -1.227643   2.0  2.786428       3311   
3  14.068408 -1.234838     3  14.068408 -1.234838   3.0  3.621876       3312   
4  14.070508 -1.221223     4  14.070508 -1.221223   4.0  3.482667       3313   

                  
            snrs  
0  3.76333286112  
1  3.17785506718  
2  2.78642836885  
3  3.62187588627  
4  3.48266666667

In [6]:
df_xcat = df.set_index('OBJID').loc[xcat[('B','OBJID')].astype(int)]
df_xcat.describe()

,RA,DEC,NH,ENERGY_SLOPE,EXPOSURE_TIME,nufnu_3keV(erg.s-1.cm-2),nufnu_error_3keV(erg.s-1.cm-2),nufnu_0.5keV(erg.s-1.cm-2),nufnu_error_0.5keV(erg.s-1.cm-2),upper_limit_0.5keV(erg.s-1.cm-2),nufnu_1.5keV(erg.s-1.cm-2),nufnu_error_1.5keV(erg.s-1.cm-2),upper_limit_1.5keV(erg.s-1.cm-2),nufnu_4.5keV(erg.s-1.cm-2),nufnu_error_4.5keV(erg.s-1.cm-2),upper_limit_4.5keV(erg.s-1.cm-2),snr
count,2764.000000,2764.000000,2.764000e+03,2764.000000,2764.000000,2.764000e+03,2.764000e+03,2.764000e+03,2.764000e+03,2.764000e+03,2.764000e+03,2.764000e+03,2.764000e+03,2.764000e+03,2.764000e+03,2.764000e+03,2764.000000
mean,15.019201,-0.106153,4.602366e+20,0.786529,35030.457308,1.325309e-13,1.134078e-14,1.228487e-13,-1.558351e-12,-9.328578e+02,1.235086e-13,-4.240899e-12,-9.256292e+02,1.478751e-13,-8.943898e-12,-8.970760e+02,6.109273
std,31.970745,0.787670,2.480186e+20,0.224127,49179.722289,1.249764e-12,1.849710e-14,1.124379e-12,8.395066e-12,2.484423e+02,1.093083e-12,2.779925e-11,2.606507e+02,1.397355e-12,4.650934e-11,3.024346e+02,10.587381
min,-58.027692,-1.639452,1.810000e+20,-1.723000,387.500000,2.048150e-15,3.481030e-16,0.000000e+00,-1.412210e-10,-9.990000e+02,0.000000e+00,-4.944210e-10,-9.990000e+02,0.000000e+00,-7.533890e-10,-9.990000e+02,2.040551
25%,-7.908507,-0.791660,2.820000e+20,0.800000,7126.325000,1.269552e-14,2.900423e-15,6.133645e-15,2.402865e-15,-9.990000e+02,8.950410e-15,3.062760e-15,-9.990000e+02,1.200200e-14,3.845992e-15,-9.990000e+02,3.102268
50%,21.870021,-0.159245,3.600000e+20,0.800000,14459.750000,2.496795e-14,6.136770e-15,1.504565e-14,5.617135e-15,-9.990000e+02,2.035930e-14,7.597640e-15,-9.990000e+02,2.716230e-14,9.770385e-15,-9.990000e+02,4.045499
75%,40.490854,0.516338,5.842500e+20,0.800000,40978.450000,5.331152e-14,1.200935e-14,3.787045e-14,1.259175e-14,-9.990000e+02,4.857688e-14,1.613043e-14,-9.990000e+02,5.982845e-14,2.051910e-14,-9.990000e+02,5.936806
max,60.102000,1.606157,1.190000e+21,3.320000,295994.000000,4.646530e-11,2.224040e-13,2.979630e-11,4.536610e-13,2.153130e-12,3.230090e-11,4.278030e-13,4.159810e-12,5.334820e-11,4.662120e-13,7.351690e-12,290.218897


In [7]:
df_xcat.head()

,RA,DEC,NH,ENERGY_SLOPE,ENERGY_SLOPE_ERROR,EXPOSURE_TIME,nufnu_3keV(erg.s-1.cm-2),nufnu_error_3keV(erg.s-1.cm-2),nufnu_0.5keV(erg.s-1.cm-2),nufnu_error_0.5keV(erg.s-1.cm-2),upper_limit_0.5keV(erg.s-1.cm-2),nufnu_1.5keV(erg.s-1.cm-2),nufnu_error_1.5keV(erg.s-1.cm-2),upper_limit_1.5keV(erg.s-1.cm-2),nufnu_4.5keV(erg.s-1.cm-2),nufnu_error_4.5keV(erg.s-1.cm-2),upper_limit_4.5keV(erg.s-1.cm-2),snr
OBJID,,,,,,,,,,,,,,,,,,
0,14.102000,-1.277310,3.540000e+20,0.8,-999/-999,4572.8,7.969610e-14,2.117700e-14,2.247330e-14,1.552740e-14,-999.0,1.088260e-13,4.027270e-14,-999.0,9.370070e-14,4.601960e-14,-999.0,3.763333
1,14.079733,-1.249499,3.520000e+20,0.8,-999/-999,4684.2,6.277090e-14,1.975260e-14,1.277270e-14,1.252210e-14,-999.0,7.075490e-14,3.430380e-14,-999.0,1.066430e-13,5.170340e-14,-999.0,3.177855
2,14.095850,-1.227643,3.510000e+20,0.8,-999/-999,4649.6,5.501970e-14,1.974560e-14,2.238590e-14,1.751890e-14,-999.0,4.655630e-14,3.005940e-14,-999.0,9.353370e-14,5.227960e-14,-999.0,2.786428
3,14.068408,-1.234838,3.510000e+20,0.8,-999/-999,4642.0,8.173270e-14,2.256640e-14,1.151060e-14,1.187720e-14,-999.0,9.568910e-14,3.936350e-14,-999.0,1.442380e-13,5.933510e-14,-999.0,3.621876
4,14.070508,-1.221223,3.500000e+20,0.8,-999/-999,4587.3,7.365840e-14,2.115000e-14,5.616880e-14,2.418840e-14,-999.0,4.673000e-14,2.649940e-14,-999.0,9.390950e-14,4.601620e-14,-999.0,3.482667


In [8]:
del df_xcat['snr']

In [9]:
detections_filename = detections_filename[:-4] + '_unique.csv'
df_xcat.to_csv(detections_filename, sep=';')

In [10]:
!head -n3 $detections_filename

OBJID;RA;DEC;NH;ENERGY_SLOPE;ENERGY_SLOPE_ERROR;EXPOSURE_TIME;nufnu_3keV(erg.s-1.cm-2);nufnu_error_3keV(erg.s-1.cm-2);nufnu_0.5keV(erg.s-1.cm-2);nufnu_error_0.5keV(erg.s-1.cm-2);upper_limit_0.5keV(erg.s-1.cm-2);nufnu_1.5keV(erg.s-1.cm-2);nufnu_error_1.5keV(erg.s-1.cm-2);upper_limit_1.5keV(erg.s-1.cm-2);nufnu_4.5keV(erg.s-1.cm-2);nufnu_error_4.5keV(erg.s-1.cm-2);upper_limit_4.5keV(erg.s-1.cm-2)
0;14.102000000000004;-1.2773102777777776;3.54e+20;0.8;-999/-999;4572.8;7.96961e-14;2.1177e-14;2.24733e-14;1.55274e-14;-999.0;1.08826e-13;4.0272699999999995e-14;-999.0;9.37007e-14;4.60196e-14;-999.0
1;14.079733333333337;-1.2494994444444445;3.52e+20;0.8;-999/-999;4684.2;6.277090000000001e-14;1.9752599999999997e-14;1.27727e-14;1.25221e-14;-999.0;7.07549e-14;3.43038e-14;-999.0;1.06643e-13;5.1703399999999997e-14;-999.0


### Countrates table

In [11]:
!head -n2 table_countrates_detections_stripe82.csv

RA;DEC;countrates_0.3-10keV(ph.s-1);countrates_error_0.3-10keV(ph.s-1);exposure_time(s);countrates_0.3-1keV(ph.s-1);countrates_error_0.3-1keV(ph.s-1);upper_limit_0.3-1keV(ph.s-1);countrates_1-2keV(ph.s-1);countrates_error_1-2keV(ph.s-1);upper_limit_1-2keV(ph.s-1);countrates_2-10keV(ph.s-1);countrates_error_2-10keV(ph.s-1);upper_limit_2-10keV(ph.s-1)
00:56:24.480;-01:16:38.317;5.645E-03;1.500E-03;4572.8;8.684E-04;6.000E-04;-9.990E+02;3.040E-03;1.125E-03;-9.990E+02;1.737E-03;8.531E-04;-9.990E+02


In [12]:
detections_filename = fname('table_countrates_detections_stripe82.csv')
flux_column = 'countrates_0.3-10keV(ph.s-1)'
flux_error_column = 'countrates_error_0.3-10keV(ph.s-1)'

import pandas
df = pandas.read_csv(detections_filename, sep=';')
df.index.name = 'OBJID'
df = df.reset_index()

xcat = run_xmatch(df, flux_column, flux_error_column)

df_xcat = df.set_index('OBJID').loc[xcat[('B','OBJID')].astype(int)]

del df_xcat['snr']

df_xcat.describe()

,RA,DEC,countrates_0.3-10keV(ph.s-1),countrates_error_0.3-10keV(ph.s-1),exposure_time(s),countrates_0.3-1keV(ph.s-1),countrates_error_0.3-1keV(ph.s-1),upper_limit_0.3-1keV(ph.s-1),countrates_1-2keV(ph.s-1),countrates_error_1-2keV(ph.s-1),upper_limit_1-2keV(ph.s-1),countrates_2-10keV(ph.s-1),countrates_error_2-10keV(ph.s-1),upper_limit_2-10keV(ph.s-1)
count,2764.000000,2764.000000,2764.000000,2764.000000,2764.000000,2764.000000,2764.000000,2764.000000,2764.000000,2764.000000,2764.000000,2764.000000,2764.000000,2764.000000
mean,15.019201,-0.106153,0.010130,0.000803,35032.097323,0.004325,-0.057754,-933.772828,0.003467,-0.118090,-926.543975,0.002740,-0.165761,-897.990529
std,31.970745,0.787670,0.090519,0.001358,49183.157865,0.037844,0.311044,249.663021,0.030600,0.780844,261.840428,0.025492,0.862769,303.547454
min,-58.027692,-1.639452,0.000119,0.000036,387.500000,0.000000,-5.195000,-2530.000000,0.000000,-13.990000,-2530.000000,0.000000,-13.990000,-2530.000000
25%,-7.908507,-0.791660,0.000854,0.000190,7126.325000,0.000219,0.000082,-999.000000,0.000246,0.000082,-999.000000,0.000223,0.000071,-999.000000
50%,21.870021,-0.159245,0.001705,0.000430,14459.750000,0.000550,0.000210,-999.000000,0.000561,0.000210,-999.000000,0.000499,0.000181,-999.000000
75%,40.490854,0.516338,0.003709,0.000830,40978.450000,0.001406,0.000469,-999.000000,0.001337,0.000444,-999.000000,0.001110,0.000385,-999.000000
max,60.102000,1.606157,2.716000,0.019000,295994.000000,1.091000,0.014000,0.082480,0.911400,0.012290,0.117300,0.944600,0.007600,0.136400


In [13]:
# df.head()

In [14]:
# xcat.head()

In [15]:
detections_filename = detections_filename[:-4] + '_unique.csv'
df_xcat.to_csv(detections_filename, sep=';')

!head -n3 $detections_filename

OBJID;RA;DEC;countrates_0.3-10keV(ph.s-1);countrates_error_0.3-10keV(ph.s-1);exposure_time(s);countrates_0.3-1keV(ph.s-1);countrates_error_0.3-1keV(ph.s-1);upper_limit_0.3-1keV(ph.s-1);countrates_1-2keV(ph.s-1);countrates_error_1-2keV(ph.s-1);upper_limit_1-2keV(ph.s-1);countrates_2-10keV(ph.s-1);countrates_error_2-10keV(ph.s-1);upper_limit_2-10keV(ph.s-1)
0;14.102000000000004;-1.2773102777777776;0.005645000000000001;0.0015;4572.8;0.0008684;0.0006;-999.0;0.0030399999999999997;0.001125;-999.0;0.001737;0.0008531000000000001;-999.0
1;14.079733333333337;-1.2494994444444445;0.004449;0.0014;4684.2;0.0004943;0.0004846;-999.0;0.001977;0.0009585;-999.0;0.001977;0.0009585;-999.0


## Checking the results

In [16]:
from bokeh.plotting import figure
from bokeh.io import output_notebook, show

output_notebook()

fig = figure()

from astropy.coordinates import Angle
r = Angle(5,'arcsec').degree

x = df_xcat['RA']
y = df_xcat['DEC']

fig.circle(x, y, fill_alpha=0.3, fill_color='blue')

show(fig)

Loading BokehJS ...